In [57]:
from sklearn import metrics

import textblob

# import src02

import model_evaluation_utils as meu

import tba3102
import pandas as pd
import numpy as np

In [58]:
tba3102.set_default_pandas_options()

In [59]:
file_name = "cleaned-galaxy_cluster_word2vec"
# file_name = "cleaned-apple_review_cluster_word2vec"
# file_name = "cleaned-samsung_vs_pixel_cluster_word2vec"
df = pd.read_csv(f'../data/results/{file_name}.csv')

In [60]:
df.head()

,Unnamed: 0,Comment ID,Comment Body,Author,Upvotes,Timestamp,Cleaned_Comment,ClusterLabel
0,0,m5u0cnr,"""Presenting, the new s25 series, now with the ...",EVD27,213,1.736231e+09,present new series new gap camera bump well di...,50
1,1,m5ttncm,"More like s24 ultra rounded version, given the...",Miyukicc,67,1.736227e+09,like ultra rounded version give minimal speck ...,22
2,2,m5tix2l,Same phone in a slightly new body (ultra) \n\n...,judgedavid90,397,1.736223e+09,phone slightly new body ultra give I shit chip...,32
3,3,m5tahhm,"Looks great, now only if they would fix their ...",One-Patience-1187,83,1.736220e+09,look great would fix camera oversaturate overe...,50
4,4,m5tut4z,The trade in deals better be out of this world,UrDoinGood2,24,1.736228e+09,trade deal well world,46


In [61]:
df["sentiment_polarity"] = [textblob.TextBlob(review).sentiment.polarity for review in df["Cleaned_Comment"]]
df["sentiments"] = ['positive' if score >= 0.3 else 'neutral' for score in df["sentiment_polarity"]]
df["sentiments"] = ['negative' if score <= -0.3 else sentiment for score,sentiment in zip(df["sentiment_polarity"],df["sentiments"])]
print(file_name)
df["sentiments"].value_counts()

cleaned-galaxy_cluster_word2vec


sentiments
neutral     378
positive     63
negative     39
Name: count, dtype: int64

In [62]:
df = df[(df["sentiments"] == "negative") | (df["sentiments"] == "positive")]
df = df.sort_values(by=["ClusterLabel","sentiments"]).reset_index(drop=True)
print(df["sentiments"].value_counts())
total = df["sentiments"].count()
pos = (df["sentiments"] == "positive").sum()
# print(pos)
print(pos/total)
print(1-pos/total)
df.head()

sentiments
positive    63
negative    39
Name: count, dtype: int64
0.6176470588235294
0.38235294117647056


,Unnamed: 0,Comment ID,Comment Body,Author,Upvotes,Timestamp,Cleaned_Comment,ClusterLabel,sentiment_polarity,sentiments
0,14,m5v8azi,The wallpapers are beautiful,UnhappyAd7832,3,1.736256e+09,wallpaper beautiful,1,0.85,positive
1,180,m6xourd,Is it only me or someone else HATE those round...,FlacFanDAC,1,1.736783e+09,I someone else hate round corner,2,-0.50,negative
2,98,m5wzs77,Thank god I'll finally be able to afford an s2...,____nothing__,1,1.736276e+09,thank god I finally able afford,2,0.50,positive
3,374,m5wkvvg,I emulate on mine and that bump in performance...,Outrageous_Tiger9921,11,1.736272e+09,I emulate mine bump performance sound amazing,2,0.50,positive
4,17,m5tcfqg,"They look awesome, can't wait for mine",JonathanDM7,6,1.736220e+09,look awesome wait mine,8,1.00,positive


In [63]:
df.to_csv(f"../data/results/{file_name}_with_sentiments.csv")

In [64]:
meu.get_metrics(true_labels=src02.test_sentiments, predicted_labels=predicted_sentiments)

NameError: name 'src02' is not defined

In [ ]:
meu.display_classification_report(true_labels=src02.test_sentiments, predicted_labels=predicted_sentiments, 
                                  classes=['negative', 'positive'])

              precision    recall  f1-score   support

    negative       0.77      0.76      0.76      7490
    positive       0.76      0.78      0.77      7510

    accuracy                           0.77     15000
   macro avg       0.77      0.77      0.77     15000
weighted avg       0.77      0.77      0.77     15000



In [ ]:
metrics.confusion_matrix(y_true=src02.test_sentiments, y_pred=predicted_sentiments, labels=['negative', 'positive'])

array([[5668, 1822],
       [1675, 5835]], dtype=int64)